In [1]:
import torch
from sidar import SIDAR

data = SIDAR("../SIDAR/", 2)

100%|██████████| 14541/14541 [00:46<00:00, 309.43it/s]


In [3]:
import torch
from torchvision import models
from torch import nn
resnet_dict = {"resnet18": models.resnet18(num_classes=128),
                            "resnet50": models.resnet50(num_classes=128)}
model = resnet_dict["resnet18"]
dim_mlp = model.fc.in_features

# add mlp projection head
model.fc = nn.Sequential(nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), model.fc)

state = torch.load("runs/Resnet18-256/checkpoint_0300.pth.tar")
state


{'epoch': 300,
 'arch': 'resnet18',
 'state_dict': OrderedDict([('conv1.weight',
               tensor([[[[ 1.7467e-02,  2.3802e-02, -4.5294e-03,  ..., -3.8817e-03,
                          -2.0841e-02, -6.7715e-03],
                         [ 9.2865e-03, -1.0288e-02,  2.6920e-02,  ..., -8.7463e-03,
                           1.3870e-02, -1.2217e-02],
                         [ 1.6889e-02, -2.3388e-02, -1.7050e-02,  ..., -3.9232e-03,
                           1.6656e-03, -2.9408e-02],
                         ...,
                         [-3.0320e-02, -1.2813e-02, -2.3244e-02,  ..., -1.0916e-02,
                           3.9936e-02,  6.4251e-03],
                         [ 1.2449e-02,  1.0966e-02, -1.6836e-03,  ...,  6.8628e-03,
                           5.6874e-03,  1.7752e-02],
                         [-2.6004e-02, -9.0185e-03, -2.5700e-02,  ..., -9.5598e-03,
                           3.7822e-02, -6.0919e-03]],
               
                        [[-1.9151e-02, -3.5468e-02

In [4]:
model.load_state_dict(state["state_dict"])


x = torch.rand(128, 3, 256, 256)
model(x).size()

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.2.weight: copying a param with shape torch.Size([256, 512]) from checkpoint, the shape in current model is torch.Size([128, 512]).
	size mismatch for fc.2.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).

In [1]:
import torch
from torchvision import models
from torch import nn
from einops import rearrange
resnet_dict = {"resnet18": models.resnet18(num_classes=128),
               "resnet34": models.resnet34(num_classes=512),
                "resnet50": models.resnet50(num_classes=1024)}

model = resnet_dict["resnet50"]
dim_mlp = model.fc.in_features
x = torch.rand(128, 3, 256, 256)
out = model(x)
out = rearrange(out, "b (c h w) -> b c h w", c=1, h=32,w=32)


model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [1]:
from models import CLRDecoder, CLREncoder
import torch

B = 84
x = torch.rand(B, 3, 256, 256).cuda()

encoder = CLREncoder("resnet34",1024).cuda()
decoder = CLRDecoder().cuda()

#decoder(encoder(x)).size()
encoder

CLREncoder(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [8]:
labels = torch.cat([torch.arange(128) for i in range(2)], dim=0)
labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
labels = labels.to(self.args.device)

In [11]:
x, y = next(iter(dataloader))   
x.size()

torch.Size([256, 2, 3, 256, 256])